# LIBRERIAS

In [5]:
!pip install pennylane

import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import math

# I. PREPROCESAMIENTO DE LOS DATOS DE ENTRADA

In [6]:
import zipfile
import os

with zipfile.ZipFile('titanic.zip', 'r') as zip_ref:
    zip_ref.extractall('titanic_data')

print("Archivos extraídos:")
print(os.listdir('titanic_data'))

df_train = pd.read_csv('/content/titanic_data/train.csv')
df_train['Pclass'] = df_train['Pclass'].astype(str)
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

Archivos extraídos:
['train.csv', 'gender_submission.csv', 'test.csv']


# II. DIVISIÓN DE LOS DATOS DE ENTRENAMIENTO Y DATOS DE PRUEBA

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train[cols_model],
    df_train['Survived'],
    test_size=0.10,
    random_state=42,
    stratify=df_train['Survived']
)

X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)

# III. ETAPA DEL FEATURE MAP

In [8]:
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, 4))

# IV. ETAPA DEL ANSATZ

In [10]:
def layer(W):
    for i in range(4):
        qml.Rot(*W[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

num_qubits = 4
num_layers = 2
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    statepreparation(x)
    for W in weights:
        layer(W)
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# V. ETAPA DE FUNCIÓN DE PERDIDA

In [11]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2
    return loss / len(labels)

# VI. ETAPA DE EVALUACIÓN DE LA FUNCIÓN DE COSTO

In [12]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# VII. ETAPA DE ANÁLISIS DE PRECISIÓN

In [13]:
def accuracy(labels, predictions):
    correct = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            correct += 1
    return correct / len(labels)

# VIII. PESOS INICIALES DEFINIDOS

In [14]:
np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

weights = weights_init
bias = bias_init
print(weights_init)
print(bias_init)

[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]]
0.0


# IX. PESOS FINALES ENTRENADOS

In [15]:
opt = AdamOptimizer(0.125)
num_it = 70
batch_size = math.floor(len(X_train) / num_it)

for it in range(num_it):
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)

    print(f"Iter: {it + 1:5d} | Cost: {cost(weights, bias, X_train, Y_train):.7f} | Accuracy: {acc:.7f}")

print(weights)
print(bias)

Iter:     1 | Cost: 2.3009054 | Accuracy: 0.3657928
Iter:     2 | Cost: 2.0157333 | Accuracy: 0.3657928
Iter:     3 | Cost: 1.6928554 | Accuracy: 0.3657928
Iter:     4 | Cost: 1.4394783 | Accuracy: 0.3657928
Iter:     5 | Cost: 1.2992964 | Accuracy: 0.5205993
Iter:     6 | Cost: 1.2592430 | Accuracy: 0.6167291
Iter:     7 | Cost: 1.2860673 | Accuracy: 0.6167291
Iter:     8 | Cost: 1.3340117 | Accuracy: 0.6167291
Iter:     9 | Cost: 1.3157367 | Accuracy: 0.6167291
Iter:    10 | Cost: 1.2278186 | Accuracy: 0.6167291
Iter:    11 | Cost: 1.1070846 | Accuracy: 0.6167291
Iter:    12 | Cost: 1.0319251 | Accuracy: 0.6167291
Iter:    13 | Cost: 0.9932298 | Accuracy: 0.6167291
Iter:    14 | Cost: 0.9745684 | Accuracy: 0.6167291
Iter:    15 | Cost: 0.9807690 | Accuracy: 0.7802747
Iter:    16 | Cost: 0.9487227 | Accuracy: 0.7802747
Iter:    17 | Cost: 0.9169272 | Accuracy: 0.7802747
Iter:    18 | Cost: 0.8685417 | Accuracy: 0.7802747
Iter:    19 | Cost: 0.8121101 | Accuracy: 0.7802747
Iter:    20 

# X. EVALUACIÓN DE DESEMPEÑO DEL CLASIFICADOR CUÁNTICO FINAL

In [16]:
X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

print("Accuracy:", accuracy_score(Y_test, predictions))
print("Precision:", precision_score(Y_test, predictions))
print("Recall:", recall_score(Y_test, predictions))
print("F1 Score:", f1_score(Y_test, predictions, average='macro'))

Accuracy: 0.7888888888888889
Precision: 0.7666666666666667
Recall: 0.6571428571428571
F1 Score: 0.7712374581939799
